# 4.5 Data Consistency

### 01. Importing libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os


In [2]:
# shortcuts for importing dataframes
path = r"C:\Users\Asus\Documents\DA CareerFoundry\Part II - Data Immersion\Python - Anaconda\August 2025 Instacart Basket Analysis\02 Data"

In [3]:
# import products.csv
df_prods = pd.read_csv(os.path.join(path, 'Original Data', 'products.csv'), index_col = False)

In [4]:
# import orders_wrangled.csv
df_ords = pd.read_csv(os.path.join(path, 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

### 02. Consistency Checks

In [5]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


Why is there an unnamed column in my describe result?
Ask tutor.

In [6]:
# create a dataframe
df_test = pd.DataFrame()

In [7]:
# create a mixed type column
df_test['mix'] = ['a', 'b', 1, True]

The first command, df_test = pd.DataFrame(), creates a new dataframe called df_test. The second command, df_test['mix'] = ['a', 'b', 1, True], creates a new column, mix, within df_test and fills it with numeric, string, and boolean values

In [8]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [9]:
# The function for checking whether a dataframe contains any mixed-type columns is as follows:
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


C:\Users\Asus\AppData\Local\Temp\ipykernel_21588\2042287563.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)


In [10]:
# with chatgpt help:
for col in df_test.columns:
    weird = (df_test[col].map(type) != type(df_test[col].iloc[0]))
    if weird.any():
        print(col)

mix


In [11]:
# converting a column type into another:
df_test['mix'] = df_test['mix'].astype('str')

Note that this will convert a column’s data type from numeric to string, but there may be times where you need to go in the opposite direction, as well—from string to numeric. To change this, simply update the str within the astype() function to int64 or whichever numeric data type you want to use.

With your mixed-type data addressed, you’re one step closer to completing your consistency checks. Next up—missing values!

### 03. Missing Values

It isn’t rare to find missing values in your data. As you learned in previous Achievements, missing values can occur for two reasons: 1) data corruption, or 2) they were never recorded in the first place. It’s important that you investigate and address any missing values in your data when conducting an analysis in Python. Similar to mixed-type columns, they can break your functions and throw errors in your analytical procedures.

In [12]:
# finding missing values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

isnull() searches and finds empty values. sum() sums them. If the latter was not used, it would show up as boolean (true, false).

There's 16 missing values in product name.

In [13]:
# creating a data frame to see where those 16 missing values are:
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [14]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


#### Ways of handling missing data

    1. Create a new variable that acts like a flag based on the missing value.
    2. Impute the value with the mean or median of the column (if the variable is numeric).
    3. Remove or filter out the missing data.


We'll remove the missing data. Here's how:

In [16]:
# number of rows and columns
df_prods.shape

(49693, 5)

In [22]:
# 16 rows in almost 50 000 is less than 1%. Let's remove them by creating a new dataframe. This time it's used " == False" because what's wanted is the non missing values.
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [23]:
df_prods_clean.shape

(49677, 5)

In [24]:
# another way of dropping the values would be: df_prods.dropna(subset = ['product_name'], inplace = True)

### 04. Finding Duplicates

In [25]:
# The following command will look for full duplicates within your dataframe:

df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [26]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


#### Addressing duplicates

In [28]:
df_prods_clean.shape

(49677, 5)

In [29]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [30]:
df_prods_clean_no_dups

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3
...,...,...,...,...,...
49688,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,5.3
49689,49685,En Croute Roast Hazelnut Cranberry,42,1,3.1
49690,49686,Artisan Baguette,112,3,7.8
49691,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,4.7


In [31]:
# less five rows.

### 05. Exporting dataframe

In [32]:
df_prods_clean_no_dups.to_csv(os.path.join(path, 'Prepared Data', 'products_checked.csv'))